## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EFC7774B20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EFC7775F90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Tejas Kamble and I am a Chief AI Engineer")])

AIMessage(content="It's nice to meet you, Tejas Kamble!  \n\nThat's a fascinating title – Chief AI Engineer.  \n\nWhat kind of work do you do in that role? I'm always eager to learn more about the exciting developments in the field of artificial intelligence.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 25, 'total_tokens': 88, 'completion_time': 0.114545455, 'prompt_time': 0.002345195, 'queue_time': 0.243398945, 'total_time': 0.11689065}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f5670b8a-eab4-4069-bc78-a885e9d7e747-0', usage_metadata={'input_tokens': 25, 'output_tokens': 63, 'total_tokens': 88})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Tejas Kamble and I am a Chief AI Engineer"),
        AIMessage(content="Hello Tejas Kamble! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="According to our conversation, your name is Tejas Kamble, and you are a Chief AI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 103, 'total_tokens': 148, 'completion_time': 0.081818182, 'prompt_time': 0.009045892, 'queue_time': 0.243216196, 'total_time': 0.090864074}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0df73b92-19c2-45d8-9426-a9a6a0a3dfac-0', usage_metadata={'input_tokens': 103, 'output_tokens': 45, 'total_tokens': 148})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Tejas and I am a Chief AI Engineer")],
    config=config
)

In [22]:
response.content

"Hello Tejas! It's a pleasure to meet you.  \n\nThat's an impressive title!  As a Chief AI Engineer, what kind of exciting projects are you working on these days?  \n\nI'm always eager to learn more about the fascinating world of AI.\n"

In [23]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Tejas.  \n\nIt's nice to officially have that confirmed!  I'm happy to address you by your correct name.  \n\nWhat can I do for you, Tejas?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 214, 'total_tokens': 260, 'completion_time': 0.083636364, 'prompt_time': 0.010410088, 'queue_time': 0.24558934199999996, 'total_time': 0.094046452}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5441296c-f996-4a2d-8e34-cec4736b4f2c-0', usage_metadata={'input_tokens': 214, 'output_tokens': 46, 'total_tokens': 260})

In [24]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You said your name is John!  \n\nDo you need help with something else, John?  \n'

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.\n\nIs there anything I can help you with today?\n"

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You told me your name is John!  \n\nIs there anything else I can help you with? 😊  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [28]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Tejas")]})

AIMessage(content="Hello Tejas, it's nice to meet you! \n\nI'm ready to answer your questions to the best of my ability. What can I help you with? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 32, 'total_tokens': 74, 'completion_time': 0.076363636, 'prompt_time': 0.003310383, 'queue_time': 0.24789308699999998, 'total_time': 0.079674019}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--43e1610d-ee44-4ec8-b89b-240dd81e4320-0', usage_metadata={'input_tokens': 32, 'output_tokens': 42, 'total_tokens': 74})

In [29]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [30]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Tejas")],
    config=config
)

response

AIMessage(content="Hello Tejas!  It's a pleasure to meet you.  \n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 274, 'total_tokens': 303, 'completion_time': 0.052727273, 'prompt_time': 0.012565685, 'queue_time': 0.245107845, 'total_time': 0.065292958}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a4ff01d5-f237-4ef0-a679-ac0fa17cf359-0', usage_metadata={'input_tokens': 274, 'output_tokens': 29, 'total_tokens': 303})

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Tejas.  😊 \n\nIs there anything else I can help you with?  \n'

In [32]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [33]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Tejas")],"language":"korean"})
response.content

'안녕하세요, 테이자스님! 반가워요. 😊 어떤 질문이 있으신가요? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [34]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [36]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Tejas")],"language":"Korean"},
    config=config
)
repsonse.content

'안녕하세요, 테자스! 😊  \n\n무엇을 도와드릴까요?  \n'

In [37]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Korean"},
    config=config,
)

In [38]:
response.content

'당신의 이름은 테자스입니다. 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    start_on="human",
    allow_partial=False,
    include_system=True
)



In [39]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\tejas\AI-ML Gen AI Projects\mlenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tejas\AI-ML Gen AI Projects\mlenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tejas\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develope

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from email import message
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

In [41]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"Korean"
    }
)
response.content

"나는 네가 어떤 아이스크림을 좋아하는지 알 수 없어요! 네가 가장 좋아하는 맛은 무엇인가요? 😊  \n\n\n(I can't know what ice cream you like! What's your favorite flavor? 😊)\n"

In [42]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "Korean",
    }
)
response.content

'2 + 2를 묻었어요! 😊  \n\n\n'

In [43]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [44]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "Korean",
    },
    config=config,
)

response.content

'죄송하지만, 저는 당신의 이름을 알지 못합니다. 당신의 이름이 뭐예요? 😊\n'

In [45]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "Korean",
    },
    config=config,
)

response.content

'죄송하지만, 과거 대화 내용을 기억할 수 없습니다. 어떤 수학 문제를 묻고 싶으신가요? 😊 \n\n'